In [21]:
# Multi Agent with supervisor
from langchain_openai.chat_models.azure import AzureChatOpenAI

import os

llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [2]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

def create_agent(llm: AzureChatOpenAI, tools: list, system_prompt:str):

    prompt = ChatPromptTemplate.from_messages(
        [
            {
                "system",
                system_prompt
            },
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools= tools)

    return executor

In [4]:
from typing import Annotated, Sequence, List, Tuple, Union
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

#tavily_tool = TavilySearchResults(max_results=5)

python_repl_tool = PythonREPLTool()

In [33]:
# wikipedia tool
import wikipedia

@tool
def wikipedia_search_tool(query: str, sentences: int = 5):
    """Searchs content in the wikipedia web site"""
    
    return wikipedia.summary(query, sentences = sentences)


In [5]:
# create retriever for RAG
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=api_base,
                api_key=api_key,
                azure_deployment=embeddings_model)

index_path = "../api/index/faiss_index"

    
vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [24]:
from typing import Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

    next: str

In [22]:
@tool
def RAG(state: AgentState):
    """Use this to execute RAG. If the questions is related to gen ai on art or music, using this tool to retrieve resuls"""

    print("Calling RAG...")

    question = state
    print("question: ", question)

    template = """Answer the question based only on the following context: {context}
    
    Question: {question}"""

    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    result = retrieval_chain.invoke(question)
    return result

In [17]:

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name = name)]}

In [48]:
# supervisor chain creation
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["RAG", "Researcher", "Coder"]

system_prompt = """You are a supervisor tasked with managing a conversation between the following workers: {members}.

Given the following user requests, respond with the worker to act next.

Use RAG tool when questions are related to GenAI in Art and GenAI in Music category.

Each worker will perform a task and respond with their results and status. 

when finished, respond with FINISH.

"""

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters":{
        "title": "routeSchema",
        "type": "object",
        "properties":{
            "next":{
                "title": "Next",
                "anyOf": [
                    {"enum": options}
                ],
            }
        },
        "required": ["next"]
    }
}


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? select one of: {options}",
        ),
    ] 
).partial(options= str(options), members=", ".join(members))

supervisor_chain = (prompt 
                    | llm.bind_functions(functions=[function_def], function_call="route")
                    | JsonOutputFunctionsParser())

In [46]:
# creation of workflow
# research node, code node, rag node
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

llm_with_tools = llm.bind_tools([wikipedia_search_tool])

research_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

research_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | research_prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

research_agent_executor = AgentExecutor(agent=research_agent, tools=[wikipedia_search_tool], verbose=True)

In [49]:
#list(research_agent_executor.stream({"input": ["Concacaf nations league"]}))

In [ ]:
import functools

research_node = functools.partial(agent_node, agent = research_agent_executor, name="Researcher")

#code_agent = create_agent(llm, 
#                          [python_repl_tool],
#                          "You may generate safe python code to analyze data and generate charts using matplotlib.")

NotImplementedError: Unsupported message type: <class 'set'>